<a href="https://colab.research.google.com/github/Sirabhop/Preclinical-AD-EEG-classification/blob/master/Machine_Learning_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import os
import pandas as pd

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
#Set working directory and import cookbook
os.chdir("/content/drive/My Drive/Colab Notebooks/")

#Download eeg feature extraction function
%run EEG_Feature_Extraction.ipynb

updated 26/4/2020, 13.28


# **Import Data**

In [0]:
os.chdir("/content/drive/My Drive/EEG Data")

In [0]:
#Compute EEG feature
subjectID = ['SS2', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8']
df_IR = extract_feature(subjectID, 'IR')

df_DL = extract_feature(subjectID, 'DL')

df = pd.concat((df_IR,df_DL), axis = 1)

#Export EEG feature
df_IR.to_csv('IR_EEG_features.csv')
df_DL.to_csv('DL_EEG_features.csv')
df.to_csv('EEG_features.csv')

In [0]:
#Import FCSRT
FCSRT = pd.read_csv("FCSRT.csv")
FCSRT.drop('id', axis = 1, inplace = True)
#Import MoCA
MoCA = pd.read_csv("MoCA.csv", usecols = ['SUM'])
#Import EEG
EEG = pd.read_csv("EEG_features.csv")
Group = pd.DataFrame(data = {'group':(1,0,1,1,0,0)})

#Create df_original
df_original = pd.concat((df_IR,df_DL), axis = 1)
df_original = pd.concat([Group,EEG,FCSRT,MoCA], axis = 1)
df_original.drop(['Unnamed: 0'], inplace = True, axis = 1)
df_original.rename(columns = {'SUM':'MoCA'}, inplace = True)
df_original.drop('id', axis = 1, inplace=True)
df_original['group'] = df_original['group'].astype('category')

df_original.head()

# **Data Preprocessing**

#### **1) Scaling**

In [0]:
from sklearn import preprocessing

In [0]:
#Min-Max Scaling

scaler = preprocessing.MinMaxScaler()
df_original[df_original.columns] = scaler.fit_transform(df_original[df_original.columns])

df_original.to_csv('Data.csv')
df_original.head()
df = df_original

#### **2) Generate Data**

The section below, I've random the value of each feature in order to demonstrate ML model because the situation of *COVID-19* that I cannot continue run the experiment as expected.

In [0]:
from random import random

In [0]:
df_random = pd.DataFrame(columns = df_original.columns)
for x in df_original.columns:
  if x != 'Group':
    df_random[x] = np.random.rand(52)
  else:
    df_random[x] = np.random.randint(0,2,52)

In [0]:
df = df_original.append(df_random, ignore_index = True)
df.tail()

,PSD_Delta,PSD_Theta,PSD_Alpha,PSD_Beta,PSD_Gamma,MSF,SE,AC,wSMI_Alpha,wSMI_Theta,IR-FreeR,IR-CuedR,DL-FreeR,DL-CuedR,MoCA,Group
55,0.807347,0.682497,0.021335,0.035987,0.381992,0.613762,0.052280,0.761692,0.513952,0.476199,0.738040,0.801613,0.066879,0.781259,0.816615,1.0
56,0.485057,0.448023,0.680089,0.031775,0.806532,0.069834,0.892653,0.272981,0.944839,0.461204,0.818933,0.112397,0.796346,0.753916,0.835964,1.0
57,0.536503,0.559438,0.170453,0.626516,0.196223,0.133042,0.470600,0.638751,0.223161,0.278665,0.705056,0.525164,0.885811,0.852113,0.109725,0.0
58,0.412441,0.072450,0.633993,0.221719,0.172257,0.631629,0.164460,0.365358,0.194858,0.917188,0.279845,0.127879,0.354827,0.744854,0.989097,0.0
59,0.379127,0.065893,0.528430,0.187522,0.183076,0.255245,0.006866,0.976507,0.282368,0.193345,0.986599,0.661297,0.992456,0.637758,0.974005,1.0


# **Model Experimentation**

####**1) Seperate dataframe**


1.   Only EEG & MoCA data
2.   FCSRT & MoCA data
3. All data


In [0]:
#1
X_1 = df[['PSD_Delta', 'PSD_Theta', 'PSD_Alpha', 'PSD_Beta', 'PSD_Gamma', 'MSF', 'SE', 'AC', 'wSMI_Alpha', 'wSMI_Theta', 'MoCA']]
#2
X_2 = df[['IR-FreeR', 'IR-CuedR','DL-FreeR', 'DL-CuedR', 'MoCA']]
#3
X_3 = df

y = df['group']

####**2) Seperate *Training Set* and *Test Set***

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
#X_1
X_1_train, X_1_test, Y_1_train,Y_1_test = train_test_split(X_1, y, test_size = 0.25, random_state = 0)

#X_2
X_2_train, X_2_test, Y_2_train, Y_2_test = train_test_split(X_2, y, test_size = 0.25, random_state = 0)

#X_3
X_3_train, X_3_test, Y_3_train, Y_3_test = train_test_split(X_3, y, test_size = 0.25, random_state = 0)

####**3) Model Building**

In [0]:
from sklearn.svm import SVC

In [0]:
classifier_1 = SVC(kernel = 'rbf', random_state = 0)
classifier_1.fit(X_1_train, Y_1_train)
Y_1_pred = classifier_1.predict(X_1_test)

classifier_2 = SVC(kernel = 'rbf', random_state = 0)
classifier_2.fit(X_2_train, Y_2_train)
Y_2_pred = classifier_2.predict(X_2_test)

classifier_3 = SVC(kernel = 'rbf', random_state = 0)
classifier_3.fit(X_3_train, Y_3_train)
Y_3_pred = classifier_3.predict(X_3_test)

####**4) Model Evaluation**

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
cm_1 = confusion_matrix(Y_1_test, Y_1_pred)
cm_2 = confusion_matrix(Y_2_test, Y_2_pred)
cm_3 = confusion_matrix(Y_3_test, Y_3_pred)